<a href="https://colab.research.google.com/github/Anjan-AI/TSAI-EVA5/blob/master/Session%204%20-%20Architectural%20Basics/EVA5_Session_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [17]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Sequential(
            
            nn.Conv2d(1, 8, 3), # output 26X26X8 : RF- 3X3
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Dropout2d(0.1),
            
            nn.Conv2d(8, 16, 3), # output 24X24X16 : : RF- 5X5
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.1),

            nn.Conv2d(16, 32, 3), # output 22X22X32 : RF- 7X7
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout2d(0.1),

            nn.Conv2d(32, 8, 1), # output 22X22X8 : RF- 7X7
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Dropout2d(0.1),
            nn.MaxPool2d(2, 2)       # output 11X11X8 : RF- 14X14
            )
        self.conv2 = nn.Sequential(
            
            nn.Conv2d(8, 16, 3), # output 09X09X16 : RF- 16 X16
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.1),
            
            nn.Conv2d(16, 32, 3), # output 07X07X32 : RF- 18 X18
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout2d(0.1),

            nn.Conv2d(32, 8, 1), # output 07X07X8 : RF- 18 X18
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Dropout2d(0.1),
           # nn.MaxPool2d(2, 2)    
            )
        self.conv3 = nn.Sequential(
           
            nn.Conv2d(8, 16, 3), # output 05X05X16 : RF- 20 X20
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.1),
            
            nn.Conv2d(16, 32, 3,padding=1), # output 05X05X32 : RF- 22 X 22
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout2d(0.1),
            nn.Dropout2d(0.1),

            nn.Conv2d(32, 10, 1), #22
           # nn.ReLU(),
           # nn.BatchNorm2d(10),

            #nn.Conv2d(16, 10, 1), #22
           # nn.BatchNorm2d(10),
            nn.AvgPool2d(5)

            )
                
        """
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(256, 512, 3)
        self.conv6 = nn.Conv2d(512, 1024, 3)
        self.conv7 = nn.Conv2d(1024, 10, 3)
        """
    def forward(self, x):
        #x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        #x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
       # x = F.relu(self.conv6(F.relu(self.conv5(x))))
        #x = self.conv7(x)
        x = self.conv1(x)
        x= self.conv2(x)
        x= self.conv3(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [18]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
              ReLU-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
         Dropout2d-4            [-1, 8, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           1,168
              ReLU-6           [-1, 16, 24, 24]               0
       BatchNorm2d-7           [-1, 16, 24, 24]              32
         Dropout2d-8           [-1, 16, 24, 24]               0
            Conv2d-9           [-1, 32, 22, 22]           4,640
             ReLU-10           [-1, 32, 22, 22]               0
      BatchNorm2d-11           [-1, 32, 22, 22]              64
        Dropout2d-12           [-1, 32, 22, 22]               0
           Conv2d-13            [-1, 8, 22, 22]             264
             ReLU-14            [-1, 8,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:89: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [19]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [20]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [23]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    print("epoch =", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    

  0%|          | 0/469 [00:00<?, ?it/s]

epoch = 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:89: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0828, Accuracy: 9767/10000 (97.67%)

epoch = 2


  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0186, Accuracy: 9940/10000 (99.40%)

epoch = 19


  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0191, Accuracy: 9937/10000 (99.37%)

epoch = 20


loss=0.020105501636862755 batch_id=468: 100%|██████████| 469/469 [00:14<00:00, 32.95it/s]



Test set: Average loss: 0.0187, Accuracy: 9936/10000 (99.36%)

